In [1]:
from AE_Util import Adversarial_Examples_Generator
from art.estimators.classification import PyTorchClassifier
from art.attacks.evasion import FastGradientMethod
import sys
sys.path.append("E:/Project/ZLTProgram/")
from Target_model import Target_model
import torch
import torch.nn as nn
from dataload import CELEBA

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# 加载目标模型
target_model=Target_model.ResNet18(2)
model_path="E:/Project/ModelAndDataset/model/CelebA/CelebA_resnet18_Male.pth"
target_model.load_state_dict(torch.load(model_path))




<All keys matched successfully>

In [3]:
from torchvision import transforms
from torch.utils.data import DataLoader
# 定义数据转换
transform=transforms.Compose([transforms.ToTensor(), transforms.RandomHorizontalFlip()])
root='E:/Project/ModelAndDataset/data'


valid_dataset = CELEBA(root=root,
               train=False,
               train_ratio=0.95,
               transform=transform,
               label="Male",
               
)
print(len(valid_dataset))


20
Load Test dataset: 10130
10130


In [4]:
estimator=PyTorchClassifier(model=target_model,loss=nn.CrossEntropyLoss(),
                            optimizer = torch.optim.Adam(target_model.parameters(), lr=0.001),
                            input_shape=(3,64,64), 
                            nb_classes=2)
#定义攻击器
attacker = FastGradientMethod(estimator, eps=0.1)


In [5]:
generator=Adversarial_Examples_Generator(targetmodel=target_model,
                                         task="Male",
                                         dataset=valid_dataset,
                                         method=attacker,
                                         targeted=False,
                                         batch_size=64,
                                         save_dir="E:/Project/ModelAndDataset/data/celebA/Attack",
                                         save_raw=True,
                                         device=device)
examplepair=generator.generate(64)

Attack Accuracy:100.00%
Save adv_img to E:/Project/ModelAndDataset/data/celebA/Attack/AdvImage_FastGradientMethod_0.1_ResNet18_Male.npy
Save adv_label to E:/Project/ModelAndDataset/data/celebA/Attack/AdvLabel_FastGradientMethod_0.1_ResNet18_Male.npy
Save raw_img to E:/Project/ModelAndDataset/data/celebA/Attack/RawImages_FastGradientMethod_0.1_ResNet18_Male.npy
Save raw_label to E:/Project/ModelAndDataset/data/celebA/Attack/RawLabels_FastGradientMethod_0.1_ResNet18_Male.npy


In [1]:
from dataload import CELEBA_Attack
from torchvision import transforms
transform=transforms.Compose([transforms.ToTensor(), transforms.RandomHorizontalFlip()])
adv_dir='E:/Project/ModelAndDataset/data/celebA/Attack'
adv_example_dataset=CELEBA_Attack(root=adv_dir,
                                  adv=True,
                                  attackMethod="FastGradientMethod",
                                  eps="0.1",
                                  model="resnet18",
                                  label="Male",
                                  transform=transform
)

Load images from: E:/Project/ModelAndDataset/data/celebA/Attack/AdvImage_FastGradientMethod_0.1_resnet18_Male.npy
Load labels from: E:/Project/ModelAndDataset/data/celebA/Attack/AdvLabel_FastGradientMethod_0.1_resnet18_Male.npy


In [2]:
type(adv_example_dataset.data)

numpy.memmap

In [3]:
import torch
mydataloader=torch.utils.data.DataLoader(adv_example_dataset)
data,label=next(iter(mydataloader))

(64, 64, 3)
<class 'numpy.memmap'>
()
<class 'numpy.float32'>


In [4]:
data

tensor([[[[0.5686, 0.5608, 0.5333,  ..., 0.8157, 0.8157, 0.8157],
          [0.6000, 0.5922, 0.5765,  ..., 0.8196, 0.8157, 0.8196],
          [0.6235, 0.6275, 0.4196,  ..., 0.8235, 0.8196, 0.8157],
          ...,
          [0.9294, 0.9255, 0.1216,  ..., 0.0196, 1.0000, 0.0471],
          [1.0000, 0.0431, 0.1216,  ..., 0.0588, 0.0275, 0.0549],
          [0.4549, 0.5098, 0.5647,  ..., 0.0275, 0.0431, 0.0275]],

         [[0.3686, 0.5608, 0.3294,  ..., 0.8275, 0.8275, 0.8275],
          [0.6039, 0.3922, 0.5765,  ..., 0.0275, 0.8275, 0.0275],
          [0.4275, 0.4314, 0.6196,  ..., 0.0314, 0.0275, 0.0235],
          ...,
          [0.0980, 0.1020, 0.1059,  ..., 0.1725, 0.9529, 1.0000],
          [0.1882, 0.2510, 0.3333,  ..., 0.0157, 0.9765, 0.9961],
          [0.4627, 0.5176, 0.5765,  ..., 0.1843, 0.1922, 0.1725]],

         [[0.3255, 0.1216, 0.1098,  ..., 0.8471, 0.8471, 0.8471],
          [0.1412, 0.1373, 0.3373,  ..., 0.8510, 0.0431, 0.0471],
          [0.1608, 0.1529, 0.1608,  ..., 0

In [5]:
label

tensor([1], dtype=torch.int32)